In [ ]:
!pip install pyspark


     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 65.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=5c793c1fe47c779446736abd8b6f13fd00d3dceaac66b991c74b319a2926008e
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
session = SparkSession.builder.appName("energy").getOrCreate()

In [ ]:
data = session.read.csv("energy_cooling.csv",inferSchema=True,header=True)

In [ ]:
data.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: double (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: integer (nullable = true)
 |-- X7: double (nullable = true)
 |-- X8: integer (nullable = true)
 |-- Y2: double (nullable = true)



In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ["X1","X2","X3","X4","X5","X6","X7","X8"],outputCol="all_features")

In [ ]:
# from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import DecisionTreeRegressor

In [ ]:
model = DecisionTreeRegressor(featuresCol="all_features",labelCol="Y2")

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
mypipeline = Pipeline(stages = [assembler,model])

In [ ]:
train,test = data.randomSplit([0.85,0.15])

In [ ]:
model = mypipeline.fit(train)

In [ ]:
results = model.transform(test)


In [ ]:
results.show(2)

+----+-----+-----+-----+---+---+---+---+-----+--------------------+------------------+
|  X1|   X2|   X3|   X4| X5| X6| X7| X8|   Y2|        all_features|        prediction|
+----+-----+-----+-----+---+---+---+---+-----+--------------------+------------------+
|0.62|808.5|367.5|220.5|3.5|  2|0.1|  4|14.37|[0.62,808.5,367.5...|14.762500000000005|
|0.62|808.5|367.5|220.5|3.5|  3|0.4|  1|17.21|[0.62,808.5,367.5...|17.126666666666665|
+----+-----+-----+-----+---+---+---+---+-----+--------------------+------------------+
only showing top 2 rows



In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
reg_eval = RegressionEvaluator(predictionCol="prediction",labelCol="Y2",metricName="rmse")
reg_eval.evaluate(results)

1.671139024682332

In [ ]:
reg_eval = RegressionEvaluator(predictionCol="prediction",labelCol="Y2",metricName="mse")
reg_eval.evaluate(results)


2.7927056398162162

In [ ]:
reg_eval = RegressionEvaluator(predictionCol="prediction",labelCol="Y2",metricName="r2")
reg_eval.evaluate(results)

0.967921661104094

In [ ]:
reg_eval = RegressionEvaluator(predictionCol="prediction",labelCol="Y2",metricName="mae")
reg_eval.evaluate(results)

1.0792340202300281